#### Objective / Problem Statement

The objective of this hackathon is to create the Machine learning predictive Model , to predict the  PM2.5 particulate
Matter concetration in the air every day for each city.

PM2.5  is the Atmospheric particulate matter that have a diameter or less that 2.5 micrometers, one of the harmful 
air pollutants.

This is Regression machine learning problem.

Evaluation Metric:
    RSME

#### Import Libraries

In [83]:
##
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from pandas import Series

##

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

##
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
##
import warnings
warnings.filterwarnings('ignore')

In [84]:
## defining the metric
def metric(x,y):
  return np.sqrt(mean_squared_error(x,y))

#### Loading Data

In [85]:
train = pd.read_csv('../Data/Train.csv')
test = pd.read_csv('../Data/Test.csv')
submission = pd.read_csv('../Data/SampleSubmission.csv')

In [86]:
train.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,29.181258,-73.896588,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,0.797294,-68.612480,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
test.head()

,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,0OS9LVX X 2020-01-02,2020-01-02,0OS9LVX,11.600000,30.200001,0.00409,14.656824,3.956377,0.712605,0.000053,...,1.445658,-95.984984,22.942019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0OS9LVX X 2020-01-03,2020-01-03,0OS9LVX,18.300001,42.900002,0.00595,15.026544,4.230430,0.661892,0.000050,...,34.641758,-95.014908,18.539116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0OS9LVX X 2020-01-04,2020-01-04,0OS9LVX,17.600000,41.299999,0.00590,15.511041,5.245728,1.640559,0.000050,...,55.872276,-94.015418,14.140820,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0OS9LVX X 2020-01-05,2020-01-05,0OS9LVX,15.011948,53.100002,0.00709,14.441858,5.454001,-0.190532,0.000055,...,59.174188,-97.247602,32.730553,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0OS9LVX X 2020-01-06,2020-01-06,0OS9LVX,9.700000,71.599998,0.00808,11.896295,3.511787,-0.279441,0.000055,...,40.925873,-96.057265,28.320527,1831.261597,3229.118652,0.031068,-100.278343,41.84708,-95.910744,28.498789


#### EDA

In [88]:
#check the shape of dataset(train,test)
train.shape , test.shape

((30557, 82), (16136, 77))

In [89]:
train.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,29.181258,-73.896588,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,0.797294,-68.612480,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
#description of train set
train.describe()

,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
count,30557.000000,30557.000000,30557.000000,3.055700e+04,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,...,23320.000000,23320.000000,23320.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000
mean,61.148045,29.025866,117.992234,7.983756e+03,125.831135,15.302326,70.552747,0.006004,9.321342,0.416886,...,35.590916,-123.697777,46.533951,923.231949,1711.793613,0.016227,1.254703,13.849040,-69.098594,23.100630
std,46.861309,33.119775,100.417713,4.863090e+04,146.581856,10.688573,18.807884,0.003787,9.343226,2.707990,...,18.955228,71.916036,14.594267,929.633988,1741.299304,0.027016,55.101250,18.004375,84.702355,24.786350
min,1.000000,1.000000,1.000000,0.000000e+00,2.000000,0.420044,5.128572,0.000139,-34.647879,-15.559646,...,0.000000,-179.880630,0.000000,0.000000,0.000000,0.000000,-105.367363,0.000000,-179.947422,0.000000
25%,25.000000,5.000000,60.000000,1.064920e+03,44.000000,7.666667,58.600002,0.003403,3.123071,-1.097864,...,19.451524,-165.882624,36.693094,0.000000,0.000000,0.000000,0.000000,0.000000,-161.726937,0.000000
50%,50.000000,15.000000,91.000000,2.395350e+03,72.000000,12.200000,74.099998,0.004912,8.478424,0.222092,...,37.918838,-156.637162,47.445010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,80.000000,44.000000,155.000000,5.882550e+03,150.000000,19.900000,85.450001,0.007562,16.201563,1.772925,...,52.270055,-118.453598,57.438181,1861.674119,3393.541633,0.023829,62.245728,27.412303,0.000000,47.090635
max,815.000000,438.000000,999.000000,1.841490e+06,1552.000000,72.599998,100.000000,0.021615,37.437921,17.955124,...,66.111289,179.776125,79.631711,2112.522949,6478.550544,0.210483,77.355232,59.972710,179.813344,69.992363


In [91]:
#check data type of train, test
train.dtypes,test.dtypes

(Place_ID X Date                                    object
 Date                                               object
 Place_ID                                           object
 target                                            float64
 target_min                                        float64
 target_max                                        float64
 target_variance                                   float64
 target_count                                        int64
 precipitable_water_entire_atmosphere              float64
 relative_humidity_2m_above_ground                 float64
 specific_humidity_2m_above_ground                 float64
 temperature_2m_above_ground                       float64
 u_component_of_wind_10m_above_ground              float64
 v_component_of_wind_10m_above_ground              float64
 L3_NO2_NO2_column_number_density                  float64
 L3_NO2_NO2_slant_column_number_density            float64
 L3_NO2_absorbing_aerosol_index                    float

In [92]:
#check on null values for train set
print(train.isnull().sum())

Place_ID X Date                                       0
Date                                                  0
Place_ID                                              0
target                                                0
target_min                                            0
target_max                                            0
target_variance                                       0
target_count                                          0
precipitable_water_entire_atmosphere                  0
relative_humidity_2m_above_ground                     0
specific_humidity_2m_above_ground                     0
temperature_2m_above_ground                           0
u_component_of_wind_10m_above_ground                  0
v_component_of_wind_10m_above_ground                  0
L3_NO2_NO2_column_number_density                   2189
L3_NO2_NO2_slant_column_number_density             2189
L3_NO2_absorbing_aerosol_index                     2189
L3_NO2_cloud_fraction                           

In [93]:
#check on null values for test set
print(test.isnull().sum())

Place_ID X Date                                       0
Date                                                  0
Place_ID                                              0
precipitable_water_entire_atmosphere                  0
relative_humidity_2m_above_ground                     0
specific_humidity_2m_above_ground                     0
temperature_2m_above_ground                           0
u_component_of_wind_10m_above_ground                  0
v_component_of_wind_10m_above_ground                  0
L3_NO2_NO2_column_number_density                   1327
L3_NO2_NO2_slant_column_number_density             1327
L3_NO2_absorbing_aerosol_index                     1327
L3_NO2_cloud_fraction                              1327
L3_NO2_sensor_altitude                             1327
L3_NO2_sensor_azimuth_angle                        1327
L3_NO2_sensor_zenith_angle                         1327
L3_NO2_solar_azimuth_angle                         1327
L3_NO2_solar_zenith_angle                       

In [94]:
# Splitting the target variable from the train dataframe
#
target = train.target

# Aligning the training and testing datasets
#
train, test = train.align(test, join = 'inner', axis = 1)


# Including a separator column to be used to split the dataframes after combining them
#
train['separator'] = 0
test['separator'] = 1


# Combining the test and train dataframes, so that feature engineering can be done on the go
#
comb = pd.concat([train, test])

# view data
comb.tail()

,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,...,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,separator
16131,ZZDJZMS X 2020-03-31,2020-03-31,ZZDJZMS,9.400001,86.200005,0.00413,3.136346,0.073416,0.923135,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16132,ZZDJZMS X 2020-04-01,2020-04-01,ZZDJZMS,17.000000,90.300003,0.00548,6.394495,-1.217649,1.270308,0.0,...,45.983723,40.416741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16133,ZZDJZMS X 2020-04-02,2020-04-02,ZZDJZMS,19.170641,86.200005,0.00656,9.750879,-0.410603,0.754363,0.0,...,-152.281488,41.202583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16134,ZZDJZMS X 2020-04-03,2020-04-03,ZZDJZMS,8.400001,66.400002,0.00457,8.322565,1.659441,-2.471616,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
16135,ZZDJZMS X 2020-04-04,2020-04-04,ZZDJZMS,6.500000,81.599998,0.00508,6.857385,-1.004568,1.752161,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [95]:
comb.shape

(46693, 78)

In [96]:
#check on null values for comb set
print(comb.isnull().sum())

Place_ID X Date                                       0
Date                                                  0
Place_ID                                              0
precipitable_water_entire_atmosphere                  0
relative_humidity_2m_above_ground                     0
specific_humidity_2m_above_ground                     0
temperature_2m_above_ground                           0
u_component_of_wind_10m_above_ground                  0
v_component_of_wind_10m_above_ground                  0
L3_NO2_NO2_column_number_density                   3516
L3_NO2_NO2_slant_column_number_density             3516
L3_NO2_absorbing_aerosol_index                     3516
L3_NO2_cloud_fraction                              3516
L3_NO2_sensor_altitude                             3516
L3_NO2_sensor_azimuth_angle                        3517
L3_NO2_sensor_zenith_angle                         3517
L3_NO2_solar_azimuth_angle                         3517
L3_NO2_solar_zenith_angle                       

In [97]:
# Converting to datetime format
comb['Date'] = pd.to_datetime(comb.Date,format='%Y-%m-%d %H:%M') 

In [98]:
# Start and end date for train
comb['Date'].min(),comb['Date'].max()

(Timestamp('2020-01-02 00:00:00'), Timestamp('2020-04-04 00:00:00'))

In [99]:
# Creating new features 
comb['day']=comb.Date.dt.day 
# weekend variable
comb['day_of_week']=comb.Date.dt.dayofweek

In [100]:
comb.tail()



,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,...,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,separator,day,day_of_week
16131,ZZDJZMS X 2020-03-31,2020-03-31,ZZDJZMS,9.400001,86.200005,0.00413,3.136346,0.073416,0.923135,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,31,1
16132,ZZDJZMS X 2020-04-01,2020-04-01,ZZDJZMS,17.000000,90.300003,0.00548,6.394495,-1.217649,1.270308,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2
16133,ZZDJZMS X 2020-04-02,2020-04-02,ZZDJZMS,19.170641,86.200005,0.00656,9.750879,-0.410603,0.754363,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3
16134,ZZDJZMS X 2020-04-03,2020-04-03,ZZDJZMS,8.400001,66.400002,0.00457,8.322565,1.659441,-2.471616,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,4
16135,ZZDJZMS X 2020-04-04,2020-04-04,ZZDJZMS,6.500000,81.599998,0.00508,6.857385,-1.004568,1.752161,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4,5


In [101]:
### Applying Kmeans to almost all features and generating a 'cluster' feature.
#comb_copy=comb.copy()
#columns=comb_copy.drop(['Place_ID X Date', 'Date', 'Place_ID'], axis=1).columns
#comb_copy=comb_copy[columns]
#km=KMeans(7,random_state=42)
#km=km.fit(comb_copy[columns])
#comb["cluster"]=km.predict(comb[columns])

In [102]:
comb["precipitable_water_entire_atmosphere"].max(),comb["precipitable_water_entire_atmosphere"].min(), comb["precipitable_water_entire_atmosphere"].mean()

(72.5999984741211, 0.4200439453125, 14.922105467722476)

In [103]:
## Dropping PLace_ID in the comb data that have more than 20
#comb = train[train['precipitable_water_entire_atmosphere']<=20]
#comb = comb[comb.index!=1094]

In [104]:
## Binned feature on precipitable_water_entire_atmosphere
#comb['precipitable_water_entire_atmospherelessthan20'] = comb['precipitable_water_entire_atmosphere'].apply(lambda x:1 if 10<x<=20  else 0)

In [105]:
comb['humiditydiff'] = np.absolute(comb['relative_humidity_2m_above_ground'] - comb['specific_humidity_2m_above_ground'])


comb['average_component_of_wind'] = np.mean(comb['u_component_of_wind_10m_above_ground'] + comb['v_component_of_wind_10m_above_ground'])
comb['sum_Aerosol'] = comb['L3_CH4_aerosol_optical_depth'] + comb['L3_CH4_aerosol_height']+comb['L3_AER_AI_absorbing_aerosol_index']
comb['stratophic'] = comb['L3_NO2_stratospheric_NO2_column_number_density'] + comb['L3_NO2_tropospheric_NO2_column_number_density']
comb["methane_angle"]= comb['L3_CH4_sensor_azimuth_angle'] + comb['L3_CH4_sensor_zenith_angle']+comb['L3_CH4_solar_azimuth_angle'] + comb['L3_CH4_solar_zenith_angle']
comb["Nitrogen_dioxide"]=comb['L3_NO2_sensor_azimuth_angle'] + comb['L3_NO2_sensor_zenith_angle'] + comb['L3_NO2_solar_azimuth_angle']+ comb['L3_NO2_solar_zenith_angle']
comb["slants"]=comb['L3_SO2_SO2_slant_column_number_density']+comb['L3_NO2_NO2_slant_column_number_density']+comb['L3_HCHO_HCHO_slant_column_number_density']
comb["sulphur_dioxide"]=comb['L3_SO2_sensor_azimuth_angle']+comb['L3_SO2_sensor_zenith_angle']+comb['L3_SO2_solar_azimuth_angle']+comb['L3_SO2_solar_zenith_angle']


In [106]:
comb.head()

,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,...,day,day_of_week,humiditydiff,average_component_of_wind,sum_Aerosol,stratophic,methane_angle,Nitrogen_dioxide,slants,sulphur_dioxide
0,010Q650 X 2020-01-02,2020-01-02,010Q650,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,...,2,3,60.191961,0.598404,3226.634718,0.000074,72.385027,75.793245,0.000105,75.740521
1,010Q650 X 2020-01-03,2020-01-03,010Q650,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,...,3,4,48.791609,0.598404,3383.159114,0.000076,96.713047,5.838171,0.000362,5.838171
2,010Q650 X 2020-01-04,2020-01-04,010Q650,16.400000,33.400002,0.00750,27.031030,5.065727,3.500559,0.000067,...,4,5,33.392502,0.598404,NaN,0.000067,NaN,-99.407349,0.000250,-99.407349
3,010Q650 X 2020-01-05,2020-01-05,010Q650,6.911948,21.300001,0.00391,23.971857,3.004001,1.099468,0.000083,...,5,6,21.296091,0.598404,NaN,0.000083,NaN,-118.504258,0.000308,-118.503951
4,010Q650 X 2020-01-06,2020-01-06,010Q650,13.900001,44.700001,0.00535,16.816309,2.621787,2.670559,0.000070,...,6,0,44.694651,0.598404,NaN,0.000070,NaN,17.934688,0.000338,17.934688


In [107]:
# Filling null Values with zero
comb['L3_CH4_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_CH4_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_CH4_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_CH4_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_CH4_aerosol_optical_depth'].fillna(0, inplace= True)
comb['L3_CH4_aerosol_height'].fillna(0, inplace= True)
comb['L3_CH4_CH4_column_volume_mixing_ratio_dry_air'].fillna(0, inplace= True)
comb['L3_SO2_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_SO2_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_SO2_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_SO2_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_SO2_cloud_fraction'].fillna(0, inplace= True)
comb['L3_SO2_absorbing_aerosol_index'].fillna(0, inplace= True)
comb['L3_SO2_SO2_slant_column_number_density'].fillna(0, inplace= True)
comb['L3_SO2_SO2_column_number_density_amf'].fillna(0, inplace= True)
comb['L3_SO2_SO2_column_number_density'].fillna(0, inplace= True)
comb['L3_AER_AI_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_AER_AI_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_AER_AI_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_AER_AI_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_AER_AI_sensor_altitude'].fillna(0, inplace= True)
comb['L3_AER_AI_absorbing_aerosol_index'].fillna(0, inplace= True)
comb['L3_CLOUD_surface_albedo'].fillna(0, inplace= True)
comb['L3_CLOUD_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_CLOUD_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_CLOUD_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_CLOUD_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_CLOUD_cloud_top_pressure'].fillna(0, inplace= True)
comb['L3_CLOUD_cloud_top_height'].fillna(0, inplace= True)
comb['L3_NO2_NO2_column_number_density'].fillna(0, inplace= True)
comb['L3_NO2_NO2_slant_column_number_density'].fillna(0, inplace= True)
comb['L3_NO2_absorbing_aerosol_index'].fillna(0, inplace= True)
comb['L3_NO2_cloud_fraction'].fillna(0, inplace= True)
comb['L3_NO2_sensor_altitude'].fillna(0, inplace= True)
comb['L3_NO2_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_NO2_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_NO2_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_NO2_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_NO2_stratospheric_NO2_column_number_density'].fillna(0, inplace= True)
comb['L3_NO2_tropopause_pressure'].fillna(0, inplace= True)
comb['L3_NO2_tropospheric_NO2_column_number_density'].fillna(0, inplace= True)
comb['L3_O3_O3_column_number_density'].fillna(0, inplace= True)
comb['L3_O3_O3_effective_temperature'].fillna(0, inplace= True)
comb['L3_O3_cloud_fraction'].fillna(0, inplace= True)
comb['L3_O3_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_O3_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_O3_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_O3_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_CO_CO_column_number_density'].fillna(0, inplace= True)
comb['L3_CO_H2O_column_number_density'].fillna(0, inplace= True)
#### Assumed to be NAN
comb['L3_CO_cloud_height'].fillna(0, inplace= True)
comb['L3_CO_sensor_altitude'].fillna(0, inplace= True)
comb['L3_CO_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_CO_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_CO_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_CO_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_HCHO_HCHO_slant_column_number_density'].fillna(0, inplace= True)
comb['L3_HCHO_cloud_fraction'].fillna(0, inplace= True)
comb['L3_HCHO_sensor_azimuth_angle'].fillna(0, inplace= True)
comb['L3_HCHO_solar_azimuth_angle'].fillna(0, inplace= True)
comb['L3_HCHO_sensor_zenith_angle'].fillna(0, inplace= True)
comb['L3_HCHO_solar_zenith_angle'].fillna(0, inplace= True)
comb['L3_HCHO_tropospheric_HCHO_column_number_density'].fillna(0, inplace= True)
comb['L3_HCHO_tropospheric_HCHO_column_number_density_amf'].fillna(0, inplace= True)
comb['L3_CLOUD_cloud_base_pressure'].fillna(0, inplace= True)
comb['L3_CLOUD_cloud_base_height'].fillna(0, inplace= True)
comb['L3_CLOUD_cloud_optical_depth'].fillna(0, inplace= True)
comb['L3_CLOUD_cloud_fraction'].fillna(0, inplace= True)
###
comb['methane_angle'].fillna(0, inplace= True)
comb['Nitrogen_dioxide'].fillna(0, inplace= True)
comb['slants'].fillna(0, inplace= True)
comb['sulphur_dioxide'].fillna(0, inplace= True)
comb['sum_Aerosol'].fillna(0, inplace= True)
comb['stratophic'].fillna(0, inplace= True)
comb['average_component_of_wind'].fillna(0, inplace= True)


In [108]:
comb['L3_SO2_solar_azimuth_angle'] = np.absolute(comb['L3_SO2_solar_azimuth_angle'])
comb['L3_CH4_solar_azimuth_angle'] = np.absolute(comb['L3_CH4_solar_azimuth_angle'])
comb['L3_O3_solar_azimuth_angle'] = np.absolute(comb['L3_O3_solar_azimuth_angle'])
comb['L3_CLOUD_solar_azimuth_angle'] = np.absolute(comb['L3_CLOUD_solar_azimuth_angle'])
comb['L3_AER_AI_solar_azimuth_angle'] = np.absolute(comb['L3_AER_AI_solar_azimuth_angle'])
comb['L3_HCHO_solar_azimuth_angle'] = np.absolute(comb['L3_HCHO_solar_azimuth_angle'])
comb['L3_NO2_solar_azimuth_angle'] = np.absolute(comb['L3_NO2_solar_azimuth_angle'])
comb['u_component_of_wind_10m_above_ground'] = np.absolute(comb['u_component_of_wind_10m_above_ground'])
comb['L3_CO_solar_azimuth_angle'] = np.absolute(comb['L3_CO_solar_azimuth_angle'])
comb['L3_CLOUD_cloud_optical_depth'] = np.absolute(comb['L3_CLOUD_cloud_optical_depth'])

##### Outliers

In [109]:
Q3, Q1 = np.percentile(comb['L3_HCHO_tropospheric_HCHO_column_number_density'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['L3_HCHO_tropospheric_HCHO_column_number_density'] = comb['L3_HCHO_tropospheric_HCHO_column_number_density'].clip(upper=whisker)

Inter quatile range:  0.00010496794265614893
Whisker            :  0.0002624198566403723


In [110]:
Q3, Q1 = np.percentile(comb['L3_HCHO_HCHO_slant_column_number_density'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['L3_HCHO_HCHO_slant_column_number_density'] = comb['L3_HCHO_HCHO_slant_column_number_density'].clip(upper=whisker)

Inter quatile range:  6.125906146834593e-05
Whisker            :  0.00015314765367086485


In [111]:
Q3, Q1 = np.percentile(comb['L3_NO2_NO2_slant_column_number_density'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['L3_NO2_NO2_slant_column_number_density'] = comb['L3_NO2_NO2_slant_column_number_density'].clip(upper=whisker)

Inter quatile range:  7.122521432627832e-05
Whisker            :  0.0002925271437824601


In [112]:
Q3, Q1 = np.percentile(comb['L3_NO2_NO2_column_number_density'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['L3_NO2_NO2_column_number_density'] = comb['L3_NO2_NO2_column_number_density'].clip(upper=whisker)

Inter quatile range:  5.66194148545421e-05
Whisker            :  0.000192902745179264


In [113]:
Q3, Q1 = np.percentile(comb['slants'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['slants'] = comb['slants'].clip(upper=whisker)

Inter quatile range:  0.0002937694440032775
Whisker            :  0.0007344236100081938


In [114]:
Q3, Q1 = np.percentile(comb['L3_NO2_tropospheric_NO2_column_number_density'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['L3_NO2_tropospheric_NO2_column_number_density'] = comb['L3_NO2_tropospheric_NO2_column_number_density'].clip(upper=whisker)

Inter quatile range:  5.386295031505491e-05
Whisker            :  0.00013465737578763728


In [115]:
Q3, Q1 = np.percentile(comb['stratophic'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['stratophic'] = comb['stratophic'].clip(upper=whisker)

Inter quatile range:  8.902388234576156e-05
Whisker            :  0.0002225597058644039


In [116]:
Q3, Q1 = np.percentile(comb['L3_CO_CO_column_number_density'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['L3_CO_CO_column_number_density'] = comb['L3_CO_CO_column_number_density'].clip(upper=whisker)

Inter quatile range:  0.014542803820544133
Whisker            :  0.0606933864610848


In [117]:
Q3, Q1 = np.percentile(comb['sum_Aerosol'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['sum_Aerosol'] = comb['sum_Aerosol'].clip(upper=whisker)

Inter quatile range:  0.0
Whisker            :  0.0


In [118]:
Q3, Q1 = np.percentile(comb['L3_CH4_CH4_column_volume_mixing_ratio_dry_air'], [75, 25])
iqr = Q3 - Q1
whisker = Q3 + (1.5*iqr)
print('Inter quatile range: ', iqr)
print('Whisker            : ', whisker)

comb['L3_CH4_CH4_column_volume_mixing_ratio_dry_air'] = comb['L3_CH4_CH4_column_volume_mixing_ratio_dry_air'].clip(upper=whisker)

Inter quatile range:  0.0
Whisker            :  0.0


In [119]:
comb["separator"].tail()

16131    1
16132    1
16133    1
16134    1
16135    1
Name: separator, dtype: int64

In [120]:
# Separating the training and testing dataframes from the combined dataframe
#
train = comb[comb.separator == 0]
test = comb[comb.separator == 1]

In [121]:
# Dropping the separator column as it has served its purpose
#
train.drop('separator', axis = 1, inplace = True)
test.drop('separator', axis = 1, inplace = True)
train['target'] = target

In [123]:
#Check correlation btn features and target
# Strong +ve correlations
post_corr=train.corr()['target'].sort_values(ascending=False)
dfcorr = pd.DataFrame(post_corr)

# display columns with Negative correlation with target
#
dfcorr.tail(5)

,target
L3_CLOUD_cloud_optical_depth,-0.107691
sulphur_dioxide,-0.119508
Nitrogen_dioxide,-0.121153
u_component_of_wind_10m_above_ground,-0.275249
L3_CH4_CH4_column_volume_mixing_ratio_dry_air,NaN


### Modelling

In [124]:
# Splitting the data into training and testing dataframes
#
#X = train.drop(['Place_ID X Date', 'Date', 'Place_ID','L3_CLOUD_cloud_optical_depth','L3_CH4_solar_azimuth_angle','L3_CLOUD_solar_azimuth_angle','L3_O3_solar_azimuth_angle','L3_AER_AI_solar_azimuth_angle','L3_HCHO_solar_azimuth_angle','L3_SO2_solar_azimuth_angle','L3_NO2_solar_azimuth_angle','u_component_of_wind_10m_above_ground','L3_CO_solar_azimuth_angle','target'], axis = 1)  # Predictors
X = train.drop(['Place_ID X Date', 'Date', 'Place_ID','L3_CH4_CH4_column_volume_mixing_ratio_dry_air','u_component_of_wind_10m_above_ground','Nitrogen_dioxide','sulphur_dioxide','L3_CLOUD_cloud_optical_depth','target'], axis = 1)  # Predictors
y = target                                                  # Target

tes = test.drop(['Place_ID X Date', 'Date','Place_ID','L3_CH4_CH4_column_volume_mixing_ratio_dry_air','u_component_of_wind_10m_above_ground','Nitrogen_dioxide','sulphur_dioxide','L3_CLOUD_cloud_optical_depth'], axis = 1)           # Testing data

In [125]:
X.shape,tes.shape

((30557, 79), (16136, 79))

In [126]:
SEED = 1

In [127]:
#kf = KFold(n_splits=5,shuffle=False)
#lgbm = XGBRegressor(n_estimators=50000,random_state=SEED,max_depth=5,learning_rate=0.05)
#scores = []
#pred_test = np.zeros(len(tes))
#for (train_index,test_index) in kf.split(X,y):
 # X_train,X_test = X.iloc[train_index],X.iloc[test_index]
 # y_train,y_test = y.iloc[train_index],y.iloc[test_index]
  #lgbm.fit(X_train,y_train,early_stopping_rounds=5,eval_set=[(X_test,y_test)],eval_metric='rmse')
 # scores.append(metric(lgbm.predict(X_test),y_test))
 # pred_test+=lgbm.predict(tes)

In [128]:
#np.mean(scores)

In [129]:
#pred_test1= np.absolute(pred_test/4)
#pred_test1

#### Submission

In [130]:
#test.target= pred_test1

In [131]:
# create submission DataFrame
#submission = pd.DataFrame({"Place_ID X Date": test["Place_ID X Date"], "target": test.target})


In [132]:
#filename = 'submission_night012.csv'
#submission.to_csv(filename,index=False)
#print('Saved file: ' + filename)

#### Catboost Model

In [144]:
predictions = []
for i in range(10):
  # Training the model with different seeds
  cat = CatBoostRegressor(verbose = False, depth = 7, iterations = 2000, random_seed = i)
  cat.fit(X, y)

  # Making predictions
  preds = cat.predict(tes)
  predictions.append(preds)

# Averaging the preictions
preds = np.mean(predictions, axis = 0)

In [145]:
preds

array([40.27420869, 33.51560819, 35.49342208, ..., 43.76035132,
       43.96342274, 49.55112625])

##### Submission

In [146]:
# Creating submission file
sub_df = pd.DataFrame({"Place_ID X Date": test["Place_ID X Date"], "target": preds})
sub_df.to_csv('submission_morn102.csv', index = False)